# Deploy model to AKS

In this example, we'll deploy a trained model to AKS. For sake of simplicty, we'll use a sample model. We'll enable authentication, automatic Swagger-interface creation, Application Insights monitoring and model data collection.

In [ ]:
!pip install azureml-sdk --upgrade
!pip install requests --upgrade

In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Environment
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model, InferenceConfig
from azureml.exceptions import ComputeTargetException

print("Azure ML SDK version:", azureml.core.VERSION)

First, we will connect to the workspace. The command `Workspace.from_config()` will either:
* Read the local `config.json` with the workspace reference (given it is there) or
* Use the `az` CLI to connect to the workspace and use the workspace attached to via `az ml folder attach -g <resource group> -w <workspace name>`

In [ ]:
ws = Workspace.from_config()
print(f'WS name: {ws.name}\nRegion: {ws.location}\nSubscription id: {ws.subscription_id}\nResource group: {ws.resource_group}')

# Preparation

Our model will be executed in an AzureML Enviromnent. We could re-use the training environment, however, if you look at the `conda.yml` in this repo, you'll see that we added a few inference-specific dependencies:

In [ ]:
env = Environment.from_conda_specification(name='credit-model-demo-environment', file_path='conda.yml')
env.register(workspace=ws)

We can directly build the environment - this will create a new Docker image in Azure Container Registry (ACR), and directly 'bake in' our dependencies from the conda definition. When we later use the Environment, all AML will need to do is pull the image for environment, thus saving the time for potentially a long-running conda environment creation.

In [ ]:
build = env.build(workspace=ws)
build.wait_for_completion(show_output=True)

Next, let's register our sample model for deployment:

In [ ]:
Model.register(model_path='model.pkl', model_name='credit-model-demo', workspace=ws)
model = Model(ws, 'credit-model-demo')

# AKS Cluster Creation

For sake of our tutorial, we'll create a small `DevTest` cluster (takes ~5 minutes). However, for real production workloads do not use flag. In many cases, you would create a cluster externally and then attach it to AzureML. If you want to provision your AKS cluster via the AzureML SDK, follow [this link](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-kubernetes?tabs=python) for more details (VNET integration, private load balancer, etc.).

In [ ]:
cluster_name = 'aks-workshop'

try:
    aks_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    
    cluster_config = {
        'cluster_purpose': AksCompute.ClusterPurpose.DEV_TEST,
        'agent_count': 1
    }
    aks_target = ComputeTarget.create(workspace=ws,
                                      name=cluster_name,
                                      provisioning_configuration=AksCompute.provisioning_configuration(**cluster_config))
    aks_target.wait_for_completion(show_output = True)

Lastly, we can kick off the model deployment:

In [ ]:
deployment_name = 'demo-model-deployment-aks'

environment = Environment.get(workspace=ws, name='credit-model-demo-environment')
inf_config = InferenceConfig(entry_script='score.py',
                             source_directory='./',
                             environment=environment)

# Full definition see https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py
aks_config = {
    'enable_app_insights': True,
    'collect_model_data': True,
    'token_auth_enabled': False,
    'auth_enabled': True,
    'cpu_cores': 1.0,
    'memory_gb': 0.5,
    'autoscale_enabled': True,
    'autoscale_min_replicas': 1,
    'autoscale_max_replicas': 2,
    'autoscale_refresh_seconds': 10,
    'autoscale_target_utilization': 70
}

aks_service = Model.deploy(workspace=ws,
                           name=deployment_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=AksWebservice.deploy_configuration(**aks_config),
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)

Let's print out some important fields from the model service:

In [ ]:
print(f"Service state: {aks_service.state}")
print(f"Service URL: {aks_service.scoring_uri}")
print(f"Service Swagger URL: {aks_service.swagger_uri}")

# Test Model

We can just fire a requests call again the model's endpoint URL, sending over the authentication key:

In [ ]:
import requests

url = aks_service.scoring_uri
key = aks_service.get_keys()[0]

test_data = {
  "data": [{
    "Age": 20,
    "Sex": "male",
    "Job": 0,
    "Housing": "own",
    "Saving accounts": "little",
    "Checking account": "little",
    "Credit amount": 100,
    "Duration": 48,
    "Purpose": "radio/TV"
  }]
}

headers = {'Content-Type': 'application/json',
           'Authorization': f'Bearer {key}'}
response = requests.post(url, json=test_data, headers=headers)

print("Prediction (good, bad):", response.text)